In [ ]:
# Установка необходимых библиотек
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 30.2 MB/

In [ ]:
# Импорт необходимых библиотек
import getpass  # для работы с паролями
import os  # для работы с окружением и файловой системой
import re  # Для работы с регулярными выражениями
import requests  # Отправка запросов
from openai import OpenAI  # Доступ к OpenAI
import gradio as gr  # Отрисовка интерфейса с помощью Gradio
import tiktoken  # Библиотека подсчёта токенов
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")


Введите OpenAI API Key:··········


In [ ]:
# Объявляем класс нейро-сотрудника
class GPT():
    def __init__(self, model="gpt-3.5-turbo"):
        self.log = ''  # атрибут для сбора логов (сообщений)
        self.model = model  # атрибут для хранения выбранной модели OpenAI
        self.search_index = None  # атрибут для хранения ссылки на базу знаний (если None, то модель не обучена)
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])  # при инициализации запрашиваем ключ от OpenAI

    def load_search_indexes(self, url):
        try:
            # Извлекаем document ID гугл документа из URL с помощью регулярных выражений
            match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
            if match_ is None:
                self.log += 'Неверный Google Docs URL\n'
                raise ValueError('Неверный Google Docs URL')
            doc_id = match_.group(1)
            export_url = f'https://docs.google.com/document/d/{doc_id}/export?format=txt'
            self.log += f'Экспортная ссылка: {export_url}\n'
            response = requests.get(export_url)
            if response.status_code != 200:
                self.log += 'Ошибка при загрузке документа\n'
                raise ValueError('Ошибка при загрузке документа')
            text = response.text
            return self.create_embedding(text)
        except Exception as e:
            self.log += f'Ошибка: {str(e)}\n'
            raise e

    def num_tokens_from_string(self, string):
        """Возвращает число токенов в строке"""
        encoding = tiktoken.encoding_for_model(self.model)  # получаем кодировщик по имени модели
        num_tokens = len(encoding.encode(string))  # расчитываем строку с помощью кодировщика
        return num_tokens  # возвращаем число токенов

    def create_embedding(self, data):
        source_chunks = []
        splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)
        for chunk in splitter.split_text(data):
            source_chunks.append(Document(page_content=chunk, metadata={}))
        count_token = self.num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
        self.log += f'Количество токенов в документе : {count_token}\n'
        self.search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )
        self.log += f'Данные из документа загружены в векторную базу данных\n'
        return self.search_index

    def num_tokens_from_messages(self, messages, model):
        """Возвращает число токенов из списка сообщений"""
        try:
            encoding = tiktoken.encoding_for_model(model)  # получаем кодировщик по имени модели
        except KeyError:
            self.log += "Предупреждение: модель не создана. Используйте cl100k_base кодировку.\n"
            encoding = tiktoken.get_encoding("cl100k_base")
        if model in {
            "gpt-3.5-turbo-0613",
            "gpt-3.5-turbo-16k-0613",
            "gpt-4-0314",
            "gpt-4-32k-0314",
            "gpt-4-0613",
            "gpt-4-32k-0613",
            "gpt-4o",
            "gpt-4o-2024-05-13"
            }:
            tokens_per_message = 3
            tokens_per_name = 1
        elif model == "gpt-3.5-turbo-0301":
            tokens_per_message = 4
            tokens_per_name = -1
        elif "gpt-3.5-turbo" in model:
            self.log += 'Внимание! gpt-3.5-turbo может обновиться в любой момент. Используйте gpt-3.5-turbo-0613.\n'
            return self.num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
        elif "gpt-4" in model:
            self.log += 'Внимание! gpt-4 может обновиться в любой момент. Используйте gpt-4-0613.\n'
            return self.num_tokens_from_messages(messages, model="gpt-4-0613")
        else:
            raise NotImplementedError(
                f"""num_tokens_from_messages() не реализован для модели {model}."""
            )

        num_tokens = 0
        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
        num_tokens += 3
        return num_tokens

    def answer_index(self, system, topic, temp=1):
        if not self.search_index:
            self.log += 'Модель необходимо обучить!\n'
            return ''
        docs = self.search_index.similarity_search(topic, k=5)
        self.log += 'Выбираем документы по степени схожести с вопросом из векторной базы данных:\n'
        message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'Отрывок документа №{i+1}:\n' + doc.page_content + '\\n' for i, doc in enumerate(docs)]))
        self.log += f'{message_content}\n'
        messages = [
            {"role": "system", "content": system + f"{message_content}"},
            {"role": "user", "content": topic}
        ]
        self.log += f"\n\nТокенов использовано на вопрос по версии TikToken: {self.num_tokens_from_messages(messages, self.model)}\n"
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=temp
        )
        self.log += 'Статистика по токенам от языковой модели:\n'
        self.log += f'Токенов использовано всего (вопрос): {completion.usage.prompt_tokens}\n'
        self.log += f'Токенов использовано всего (вопрос-ответ): {completion.usage.total_tokens}\n'
        return completion.choices[0].message.content

    def structure_document(self, url):
        try:
            # Извлекаем document ID гугл документа из URL с помощью регулярных выражений
            match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
            if match_ is None:
                self.log += 'Неверный Google Docs URL\n'
                raise ValueError('Неверный Google Docs URL')
            doc_id = match_.group(1)
            export_url = f'https://docs.google.com/document/d/{doc_id}/export?format=txt'
            self.log += f'Экспортная ссылка: {export_url}\n'
            response = requests.get(export_url)
            if response.status_code != 200:
                self.log += 'Ошибка при загрузке документа\n'
                raise ValueError('Ошибка при загрузке документа')
            text = response.text
            # Запрос к модели для структурирования документа
            messages = [
                {"role": "system", "content": "Ты эксперт по структурированию документов. Структурируй следующий текст:"},
                {"role": "user", "content": text}
            ]
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.7
            )
            structured_text = completion.choices[0].message.content
            self.log += f'Документ структурирован\n'
            return structured_text
        except Exception as e:
            self.log += f'Ошибка: {str(e)}\n'
            raise e


In [ ]:
# Создаем список нейро-сотрудников с новой ссылкой на документ
models = [
    {
        "doc": "https://docs.google.com/document/d/1_Diwb5QCCxj1v0abrZ8MRUHA8nr5SlrJrorDJ-AP8zs/export?format=txt",
        "prompt": '''Ты IT-специалист, к которому могут обращаться другие IT-специалисты за помощью в решении технических вопросов и задач.
                     Дай развернутый и точный ответ на их вопросы, предоставь пошаговые инструкции, если это необходимо.
                     Не добавляй лишних эмоций и слов, от тебя нужна только точная информация.
                     Отвечай максимально точно по документу, не придумывай ничего от себя.
                     Документ с информацией для ответа клиенту:''',
        "name": "Нейро-помощник IT-специалиста",
        "query": "Как настроить сетевое подключение на Ubuntu?"
    }
]


In [ ]:
# Gradio интерфейс
gpt = GPT("gpt-3.5-turbo")

blocks = gr.Blocks()

with blocks as demo:
    subject = gr.Dropdown([(elem["name"], index) for index, elem in enumerate(models)], label="Данные")
    name = gr.Label(show_label=False)
    prompt = gr.Textbox(label="Промт", interactive=True)
    link = gr.HTML()
    query = gr.Textbox(label="Запрос к LLM", interactive=True)

    def onchange(dropdown):
        return [
            models[dropdown]['name'],
            re.sub('\t+|\s\s+', ' ', models[dropdown]['prompt']),
            models[dropdown]['query'],
            f"<a target='_blank' href = '{models[dropdown]['doc']}'>Документ для обучения</a>"
        ]

    subject.change(onchange, inputs=[subject], outputs=[name, prompt, query, link])

    with gr.Row():
        structure_btn = gr.Button("Структурировать документ")
        train_btn = gr.Button("Обучить модель")
        request_btn = gr.Button("Запрос к модели")

    def structure(dropdown):
        try:
            if dropdown is None:
                return "Ошибка: Не выбран нейро-сотрудник"
            structured_text = gpt.structure_document(models[dropdown]['doc'])
            return structured_text
        except Exception as e:
            return f"Ошибка при структурировании документа: {str(e)}"

    def train(dropdown):
        try:
            if dropdown is None:
                return "Ошибка: Не выбран нейро-сотрудник"
            gpt.load_search_indexes(models[dropdown]['doc'])
            return gpt.log
        except Exception as e:
            return f"Ошибка при обучении модели: {str(e)}"

    def predict(p, q):
        result = gpt.answer_index(
            p,
            q
        )
        return [result, gpt.log]

    with gr.Row():
        response = gr.Textbox(label="Ответ LLM")
        log = gr.Textbox(label="Логирование")

    structure_btn.click(structure, [subject], log)
    train_btn.click(train, [subject], log)
    request_btn.click(predict, [prompt, query], [response, log])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6d3cd0d054fe02103c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1. Структурирование документа:
   Я использовал разделы в Google Документе, чтобы структурировать информацию и упростить её поиск. Это поможет улучшить точность и релевантность ответов нейро-сотрудника.

2. Оптимизация:
   - Разбиение текста на чанки позволило улучшить обработку и индексирование данных.
   - Подсчет токенов помог оптимизировать затраты на запросы к модели.
   - Тестирование различных запросов помогло отладить и улучшить качество ответов нейро-сотрудника.

Эти шаги были предприняты для повышения эффективности и точности работы нейро-сотрудника, что позволит лучше решать задачи пользователей и предоставлять более точные ответы на их запросы.
